# PersonReID

In [5]:
import json
import pandas as pd

def get_labels(json_data):
  df = pd.DataFrame(columns=json_data.keys())
  for key in json_data.keys():
    df[key] = json_data[key]
  df.set_index('image_index', inplace=True)
  df.sort_index(inplace=True)
  df.index = df.index.astype(int)
  df = df-1
  df.loc[df['age'] == 1, 'age'] = 0
  df.loc[df['age'] > 1, 'age'] = 1
  return df.sort_index(axis=1)

def get_train_test_labels():
  with open('market_attribute.json') as json_file:
    data = json.load(json_file)
    train = data['train']
    test = data['test']
  return get_labels(train), get_labels(test)

In [6]:
#GetData
import cv2
import glob
# import image and preprocess_input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
import numpy as np
import pandas as pd

HEIGHT = 128
WIDTH = 64




def BuildData(labelsTab, dataPath):
    dataTab = []
    files = glob.glob (dataPath)
    imageID = 0
    for myFile in files:
        if(imageID in labelsTab.index):
            img = cv2.imread(myFile)
            dataTab.append(img)
        imageID += 1


    print('X_data shape:', np.array(dataTab).shape)
    dataTab = np.array(dataTab)
    return dataTab

def PreprocessImages(dataTab):
    for img in dataTab:
        img=preprocess_input(img)
    return dataTab


train_labels, test_labels = get_train_test_labels()
train_labels_columns = train_labels.columns
test_labels_columns = test_labels.columns


dataPath = "Market-1501/*.jpg"

train_data = BuildData(train_labels, dataPath)
test_data = BuildData(test_labels, dataPath)

train_data = PreprocessImages(train_data)
test_data = PreprocessImages(test_data)    

train_labels = train_labels.values
test_labels = test_labels.values

X_data shape: (751, 128, 64, 3)
X_data shape: (750, 128, 64, 3)


In [7]:
# import ResNet50 and decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import BatchNormalization


input_shape = (HEIGHT, WIDTH, 3)
# instantiate a ResNet50 model with imagenet weights
base_model=ResNet50(weights='imagenet', include_top=False, 
                      input_shape=input_shape)

####
#Add layers

output = base_model.layers[-1].output
output = Flatten()(output)

resnet = Model(base_model.input, outputs=output)

# Test without training resnet weights again

#for layer in restnet.layers:
    #layer.trainable = False
    
#####
dropout = 0.5

# define new model
model = Sequential()
model.add(resnet)
model.add(Dense(512, activation='relu', input_dim=input_shape))
model.add(BatchNormalization())
model.add(Dropout(dropout))
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(train_labels.shape[1], activation='sigmoid'))

#VOIR LES LAYERS EXACT SUR LE PAPER
#####

NUM_EPOCHS = 60
BATCH_SIZE = 32
num_train_images = len(train_data)

#To change learning rate during training like in paper :
def step_decay(epoch):
    lrate = 0.01
    if(epoch >= NUM_EPOCHS-21): 
        lrate = 0.001
    print("lrate = " + str(lrate))
    return lrate





adam = Adam(lr=0.01)
model.compile(adam, loss='binary_crossentropy', metrics=['accuracy'])

lrate = LearningRateScheduler(step_decay)

filepathNoAugmentation="./checkpoints/" + "ResNet50" + "_model_weights_noAugmentation.h5"
filepathAugmentation="./checkpoints/" + "ResNet50" + "_model_weights_augmentation.h5"
checkpointNoAugmentation = ModelCheckpoint(filepathNoAugmentation, monitor=["acc"], verbose=1, mode='max')
checkpointAugmentation = ModelCheckpoint(filepathAugmentation, monitor=["acc"], verbose=1, mode='max')
callbacks_list_noAugmentation = [checkpointNoAugmentation, lrate]
callbacks_list_augmentation = [checkpointAugmentation, lrate]

In [8]:
history = model.fit(train_data,train_labels, epochs=NUM_EPOCHS,
                                       batch_size = BATCH_SIZE,
                                       validation_data = (test_data, test_labels),
                                       shuffle=True, callbacks=callbacks_list_noAugmentation)

Train on 751 samples, validate on 750 samples
lrate = 0.01
Epoch 1/60
736/751 [============================>.] - ETA: 1s - loss: 0.6151 - accuracy: 0.7348
Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 105s 139ms/sample - loss: 0.6142 - accuracy: 0.7366 - val_loss: 1179264437766673.7500 - val_accuracy: 0.7727
lrate = 0.01
Epoch 2/60
736/751 [============================>.] - ETA: 1s - loss: 0.4412 - accuracy: 0.8359
Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 94s 125ms/sample - loss: 0.4412 - accuracy: 0.8359 - val_loss: 604975198808.7467 - val_accuracy: 0.8042
lrate = 0.01
Epoch 3/60
736/751 [============================>.] - ETA: 1s - loss: 0.4488 - accuracy: 0.8369
Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 95s 127ms/sample - loss: 0.4485 -

751/751 [==============================] - 107s 142ms/sample - loss: 0.3712 - accuracy: 0.8477 - val_loss: 0.5110 - val_accuracy: 0.8443
lrate = 0.01
Epoch 26/60
736/751 [============================>.] - ETA: 1s - loss: 0.3718 - accuracy: 0.8465
Epoch 00026: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 105s 140ms/sample - loss: 0.3720 - accuracy: 0.8465 - val_loss: 0.4839 - val_accuracy: 0.8404
lrate = 0.01
Epoch 27/60
736/751 [============================>.] - ETA: 1s - loss: 0.3696 - accuracy: 0.8481
Epoch 00027: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 98s 131ms/sample - loss: 0.3691 - accuracy: 0.8481 - val_loss: 0.4109 - val_accuracy: 0.8461
lrate = 0.01
Epoch 28/60
736/751 [============================>.] - ETA: 1s - loss: 0.3697 - accuracy: 0.8470
Epoch 00028: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 

751/751 [==============================] - 111s 148ms/sample - loss: 0.3608 - accuracy: 0.8493 - val_loss: 0.3739 - val_accuracy: 0.8514
lrate = 0.001
Epoch 51/60
736/751 [============================>.] - ETA: 1s - loss: 0.3615 - accuracy: 0.8494
Epoch 00051: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 104s 139ms/sample - loss: 0.3614 - accuracy: 0.8491 - val_loss: 0.3735 - val_accuracy: 0.8507
lrate = 0.001
Epoch 52/60
736/751 [============================>.] - ETA: 1s - loss: 0.3606 - accuracy: 0.8497
Epoch 00052: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/751 [==============================] - 104s 138ms/sample - loss: 0.3605 - accuracy: 0.8495 - val_loss: 0.3740 - val_accuracy: 0.8507
lrate = 0.001
Epoch 53/60
736/751 [============================>.] - ETA: 1s - loss: 0.3606 - accuracy: 0.8495
Epoch 00053: saving model to ./checkpoints/ResNet50_model_weights_noAugmentation.h5
751/

In [9]:
#RexecHere
#Data augmentation
from keras.preprocessing.image import ImageDataGenerator

def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)



train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3,
                                  rotation_range=0.2, height_shift_range=0.2,
                                  shear_range=0.2,
                                  horizontal_flip=True, fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_data, train_labels, BATCH_SIZE)

val_generator = val_datagen.flow(test_data, test_labels, BATCH_SIZE)

train_generator = crop_generator(train_generator, 224)
val_generator = crop_generator(val_generator, 224)

train_generator = crop_generator

Using TensorFlow backend.


In [10]:
history_augmentedData = model.fit(train_generator, epochs = NUM_EPOCHS,
                                 validation_data = val_generator,
                                 shuffle = True, callbacks=callbacks_list_augmentation)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 24 steps, validate for 24 steps
lrate = 0.01
Epoch 1/60
23/24 [===========================>..] - ETA: 3s - loss: 0.3681 - accuracy: 0.8469
Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights_augmentation.h5
24/24 [==============================] - 104s 4s/step - loss: 0.3676 - accuracy: 0.8477 - val_loss: 0.3603 - val_accuracy: 0.8534
lrate = 0.01
Epoch 2/60
23/24 [===========================>..] - ETA: 3s - loss: 0.3669 - accuracy: 0.8485
Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights_augmentation.h5
24/24 [==============================] - 93s 4s/step - loss: 0.3673 - accuracy: 0.8482


KeyboardInterrupt: 

In [17]:
# predictions without augmented data
from tensorflow.keras.models import load_model

resultModel = load_model("checkpoints/ResNet50_model_weights_noAugmentation.h5")

img_index = 20

img = test_data[img_index]
actualLabels = test_labels[img_index]
img = np.array(img)
img=preprocess_input(img)

img = img.reshape(1,img.shape[0], img.shape[1], img.shape[2])

preds=resultModel.predict(img)
print("##################")
print("Model predictions : ")

print(preds)
print("##################")
print("")
print("##################")
print("Actual labels : ")
print(actualLabels)
print("##################")


print("model accuracy :")
print(resultModel.evaluate(test_data,test_labels))

##################
Model predictions : 
[[0.23870702 0.2732263  0.26119956 0.81937164 0.56237483 0.4625576
  0.14547735 0.06759659 0.1562219  0.01007827 0.03591957 0.00309933
  0.08223568 0.0077389  0.5030342  0.36876315 0.12835479 0.02327012
  0.952761   0.19745387 0.05936061 0.10761686 0.07718932 0.04099803
  0.07834872 0.32725444 0.02491984]]
##################

##################
Actual labels : 
[0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
##################
model accuracy :
750/750 [==============================] - 12s 17ms/sample - loss: 0.4340 - accuracy: 0.8443
[0.4340207044283549, 0.8442963]


In [12]:
# predictions with augmented data
from tensorflow.keras.models import load_model
resultModel = load_model("checkpoints/ResNet50_model_weights_augmentation.h5")

img_index = 450

img = test_data[img_index]
actualLabels = test_labels[img_index]
img = np.array(img)
img=preprocess_input(img)

img = img.reshape(1,img.shape[0], img.shape[1], img.shape[2])

preds=resultModel.predict(img)
print("##################")
print("Model predictions : ")

print(preds)
print("##################")
print("")
print("##################")
print("Actual labels : ")
print(actualLabels)
print("##################")


print("model accuracy :")
print(resultModel.evaluate(test_data,test_labels))

OSError: SavedModel file does not exist at: checkpoints/ResNet50_model_weights_augmentation.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
#Comparison between two features vectors
personA = np.array([0.5, 0.8, 0.9])
personB = np.array([0.8, 0.2, 0.6])
personC = np.array([])
dist = np.linalg.norm(personA - personB)
dist2 = np.linalg.norm(personA - personB)

print(dist)
print(dist2)

In [ ]:
plot_training(history)

# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')